In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 50      # number of variables
num_ineq = 50     # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

## Heuristic - Round

In [7]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [05:38<00:00,  2.95it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -16.141183          1.883578      0.245323
std       0.318926          0.487851      0.069632
min     -16.979353          0.470149      0.156702
25%     -16.366533          1.517763      0.199059
50%     -16.138196          1.847186      0.231558
75%     -15.916087          2.209809      0.272832
max     -15.202193          3.384434      0.848458
Number of infeasible solution: 1000


## Heuristic - N1

In [9]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [10]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_n1_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:14<00:00,  3.93it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.587427e+17               0.0      0.243883
std    1.495455e+17               0.0      0.111663
min   -1.628116e+01               0.0      0.157912
25%   -1.098950e+01               0.0      0.193062
50%    2.266936e+17               0.0      0.224638
75%    2.994127e+17               0.0      0.268512
max    5.661554e+17               0.0      2.894798
Number of infeasible solution: 0


### Heuristic - N3

In [11]:
model_heur = model.first_solution_heuristic(nodes_limit=3)

In [12]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_n3_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:40<00:00,  2.50it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.467174e+16               0.0      0.390463
std    5.131523e+16               0.0      1.107813
min   -1.657508e+01               0.0      0.163690
25%   -1.536716e+01               0.0      0.223936
50%   -1.346294e+01               0.0      0.276416
75%    2.006904e+12               0.0      0.375336
max    2.797341e+17               0.0     30.982206
Number of infeasible solution: 0


## Learnable Rounding

In [13]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [14]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [15]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [16]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 742.94
Epoch 1, Validation Loss: -7.44
Epoch 2, Validation Loss: -10.35
Epoch 3, Validation Loss: -10.25
Epoch 4, Validation Loss: -10.97
Epoch 5, Validation Loss: -10.17
Epoch 6, Validation Loss: -11.10
Epoch 7, Validation Loss: -10.93
Epoch 8, Validation Loss: -10.79
Epoch 9, Validation Loss: -11.34
Epoch 10, Validation Loss: -10.98
Epoch 11, Validation Loss: -11.13
Epoch 12, Validation Loss: -11.22
Epoch 13, Validation Loss: -11.23
Epoch 14, Validation Loss: -11.65
Epoch 15, Validation Loss: -11.80
Epoch 16, Validation Loss: -11.77
Epoch 17, Validation Loss: -11.52
Epoch 18, Validation Loss: -11.70
Epoch 19, Validation Loss: -11.49
Epoch 20, Validation Loss: -11.61
Epoch 21, Validation Loss: -11.52
Epoch 22, Validation Loss: -11.68
Epoch 23, Validation Loss: -11.71
Epoch 24, Validation Loss: -11.47
Epoch 25, Validation Loss: -11.93
Epoch 26, Validation Loss: -11.73
Epoch 27, Validation Loss: -11.85
Epoch 28, Validation Loss: -12.24
Epoch 29, Validation Loss

In [17]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:17<00:00, 58.69it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -13.569813          0.003140      0.004451
std       0.295833          0.020386      0.001758
min     -14.394269          0.000000      0.001997
25%     -13.785014          0.000000      0.003008
50%     -13.568596          0.000000      0.004001
75%     -13.371199          0.000000      0.005297
max     -12.658380          0.278483      0.012526
Number of infeasible solution: 44


## Learnable Threshold

In [18]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [19]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [20]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [21]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 620.59
Epoch 1, Validation Loss: -8.30
Epoch 2, Validation Loss: -11.27
Epoch 3, Validation Loss: -11.59
Epoch 4, Validation Loss: -11.44
Epoch 5, Validation Loss: -11.75
Epoch 6, Validation Loss: -11.97
Epoch 7, Validation Loss: -11.92
Epoch 8, Validation Loss: -11.41
Epoch 9, Validation Loss: -11.73
Epoch 10, Validation Loss: -12.00
Epoch 11, Validation Loss: -12.00
Epoch 12, Validation Loss: -12.27
Epoch 13, Validation Loss: -11.40
Epoch 14, Validation Loss: -12.32
Epoch 15, Validation Loss: -12.07
Epoch 16, Validation Loss: -11.99
Epoch 17, Validation Loss: -12.13
Epoch 18, Validation Loss: -12.10
Epoch 19, Validation Loss: -12.43
Epoch 20, Validation Loss: -11.87
Epoch 21, Validation Loss: -12.20
Epoch 22, Validation Loss: -12.37
Epoch 23, Validation Loss: -12.01
Epoch 24, Validation Loss: -12.07
Epoch 25, Validation Loss: -11.80
Epoch 26, Validation Loss: -12.37
Epoch 27, Validation Loss: -12.51
Epoch 28, Validation Loss: -12.44
Epoch 29, Validation Loss

In [22]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:17<00:00, 56.29it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -13.397122          0.004460      0.004476
std       0.274324          0.026725      0.001672
min     -14.132930          0.000000      0.001537
25%     -13.585816          0.000000      0.003274
50%     -13.419505          0.000000      0.004050
75%     -13.220037          0.000000      0.005211
max     -12.211704          0.350001      0.014135
Number of infeasible solution: 55
